In [76]:
from __future__ import print_function

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from langdetect import detect
from langdetect import detect_langs

In [78]:
pd.set_option('display.max_columns', None)

In [79]:
buttigieg = pd.read_csv('buttigieg_cleaned.csv')

warren = pd.read_csv('warren_cleaned.csv')

bernie = pd.read_csv('bernie_cleaned.csv')

biden = pd.read_csv('biden_cleaned.csv')

amy = pd.read_csv('klobuchar_cleaned.csv')

bloomberg = pd.read_csv('bloomberg_cleaned.csv')

trump = pd.read_csv('trump_cleaned.csv')

In [80]:
all_tweets = list(buttigieg['tweet']) + list(warren['tweet']) + list(bernie['tweet']) + list(biden['tweet']) + list(amy['tweet']) + list(bloomberg['tweet']) + list(trump['tweet'])

In [93]:
nltk.download('stopwords')
stop_words=set(stopwords.words("english"))
stop_words=(stopwords.words("english"))
political_stopwords = ['mikebloomberg', 'ewarren', 'senwarren', 'berniesanders', 'sensanders', 'joebiden', 'mike', 'michael', 'cream', 'pony',
                       'petebuttigieg', 'sentinasmith', 'sen', 'amyklobuchar', 'peter', 'donaldtrump', 'realdonaldtrump', 'oscar', 'oscars',
                       'madoff', 'taupin', 'joseph', 'i', 'says', 'yang', 'steyer', 'tom', 'andrew', 'elizabeth', 'joe', 'donald', 'medal',
                      'song', 'elton', 'john', 'rocketman', 'mitt', 'romney', 'ponzi', 'kobe', 'news', 'wine', 'meatball', 'sexually', 'original',
                      'best', 'diane', 'palestinian', 'every', 'alexander', 'rush', 'limbaugh', 'vindman', 'said', 'say', 'says', 'mary', 'winstead',
                      'people', 'like', 'think', 'one', 'would', 'get', 'know', 'want', 'going', 'also', 'even', 'voters', 'votes', 'new', 'need',
                      'year', 'take', 'good', 'tonight', 'see', 'still', 'back', 'big', 'night', 'day', 'love', 'first', 'last', 'could', 'time',
                      'thing', 'city', 'town', 'hall', 'care', 'right', 'long', 'well', 'country', 'make', 'kamala', 'harris', 'dog', 'faced',
                      'soldier', 'bro', 'judy', 'super', 'mini', 'superbowl']
for i in range(len(political_stopwords)):
    stop_words.append(political_stopwords[i])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Robert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                ngram_range = (1,2),
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(all_tweets)
print(dtm_tf.shape)

(90824, 17335)


In [95]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(all_tweets)
print(dtm_tfidf.shape)

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1817: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(90824, 17335)


In [96]:
lda_tf = LatentDirichletAllocation(n_components=7, random_state=0)
tf_fit = lda_tf.fit(dtm_tf)
tf_fit

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [107]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_tf.score(dtm_tf))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_tf.perplexity(dtm_tf))

Log Likelihood:  -9794002.038919557
Perplexity:  3899.3578245044473


In [97]:
lda_tfidf = LatentDirichletAllocation(n_components=7, random_state=0)
tfidf_fit = lda_tfidf.fit(dtm_tfidf)
tfidf_fit

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [105]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_tfidf.score(dtm_tfidf))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_tfidf.perplexity(dtm_tfidf))

Log Likelihood:  -2873028.5040659616
Perplexity:  15716.662230788423
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 7, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': 0, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [101]:
tf_model = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
tf_model

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.011204 -0.097618       1        1  22.811980
0     -0.103646  0.016255       2        1  14.895945
5     -0.119843 -0.074819       3        1  14.868084
2      0.084517  0.136168       4        1  14.123763
3     -0.084309 -0.139702       5        1  13.302649
6     -0.052813  0.228008       6        1  12.358246
1      0.287298 -0.068293       7        1   7.639333, topic_info=      Category          Freq       Term         Total  loglift  logprob
1809   Default  16638.000000  bloomberg  16638.000000  30.0000  30.0000
11001  Default   8779.000000       pete   8779.000000  29.0000  29.0000
15539  Default  20220.000000      trump  20220.000000  28.0000  28.0000
1270   Default  11043.000000     bernie  11043.000000  27.0000  27.0000
2453   Default   6219.000000  buttigieg   6219.000000  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
11737   Topic7    240.553525        pro    469.530003   1.9031  -5.9300
8006    Topic7    252.520498       iran    667.471375   1.5999  -5.8815
11523   Topic7    359.153315  president   7785.355961  -0.5044  -5.5292
11277   Topic7    262.997045     policy   1085.210645   1.1545  -5.8408
1270    Topic7    302.204469     bernie  11043.419366  -1.0266  -5.7019

[523 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6          4  0.990922        abbas
27         5  0.997566     abortion
30         5  0.978944    abortions
46         4  0.994459        abuse
47         4  0.995824  abuse power
...      ...       ...          ...
17299      4  0.018099         york
17299      5  0.056881         york
17299      6  0.620522         york
17299      7  0.040075         york
17300      6  0.996875   york mayor

[1269 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 3, 4, 7, 2])

In [100]:
tfidf_model = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)
tfidf_model

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.051414 -0.114295       1        1  17.705413
2     -0.001784 -0.048840       2        1  16.959049
5      0.096237  0.043370       3        1  16.442657
6      0.019202  0.170474       4        1  14.427275
3      0.037369 -0.024629       5        1  12.000591
0      0.037457 -0.025217       6        1  11.608748
1     -0.239897 -0.000863       7        1  10.856266, topic_info=      Category         Freq            Term        Total  loglift  logprob
9402   Default   813.000000           mayor   813.000000  30.0000  30.0000
1270   Default  1514.000000          bernie  1514.000000  29.0000  29.0000
11001  Default  1240.000000            pete  1240.000000  28.0000  28.0000
9417   Default   596.000000      mayor pete   596.000000  27.0000  27.0000
14393  Default   284.000000           stone   284.000000  26.0000  26.0000
...        ...          ...             ...          ...      ...      ...
1430    Topic7   107.057442           biden  1415.955893  -0.3618  -5.7088
187     Topic7    67.439527  administration   177.980808   1.2500  -6.1710
16633   Topic7    86.594228          warren  1158.542528  -0.3733  -5.9210
1809    Topic7    85.753990       bloomberg  2003.095012  -0.9305  -5.9307
1270    Topic7    78.382623          bernie  1514.617301  -0.7409  -6.0206

[557 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
6          1  0.049144          abbas
6          2  0.933735          abbas
27         2  0.045042       abortion
27         4  0.165155       abortion
27         5  0.780735       abortion
...      ...       ...            ...
17184      1  0.192824  working class
17184      2  0.728447  working class
17184      3  0.042850  working class
17184      5  0.021425  working class
17184      6  0.021425  working class

[1427 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 6, 7, 4, 1, 2])

In [99]:
mmds_model = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')
mmds_model

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.074738 -0.175955       1        1  22.811980
0      0.033166  0.226526       2        1  14.895945
5     -0.195122  0.204682       3        1  14.868084
2     -0.192482 -0.040112       4        1  14.123763
3      0.232652 -0.090731       5        1  13.302649
6      0.216714  0.164977       6        1  12.358246
1     -0.169666 -0.289387       7        1   7.639333, topic_info=      Category          Freq       Term         Total  loglift  logprob
1809   Default  16638.000000  bloomberg  16638.000000  30.0000  30.0000
11001  Default   8779.000000       pete   8779.000000  29.0000  29.0000
15539  Default  20220.000000      trump  20220.000000  28.0000  28.0000
1270   Default  11043.000000     bernie  11043.000000  27.0000  27.0000
2453   Default   6219.000000  buttigieg   6219.000000  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
11737   Topic7    240.553525        pro    469.530003   1.9031  -5.9300
8006    Topic7    252.520498       iran    667.471375   1.5999  -5.8815
11523   Topic7    359.153315  president   7785.355961  -0.5044  -5.5292
11277   Topic7    262.997045     policy   1085.210645   1.1545  -5.8408
1270    Topic7    302.204469     bernie  11043.419366  -1.0266  -5.7019

[523 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6          4  0.990922        abbas
27         5  0.997566     abortion
30         5  0.978944    abortions
46         4  0.994459        abuse
47         4  0.995824  abuse power
...      ...       ...          ...
17299      4  0.018099         york
17299      5  0.056881         york
17299      6  0.620522         york
17299      7  0.040075         york
17300      6  0.996875   york mayor

[1269 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 3, 4, 7, 2])

In [98]:
tsne_model = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')
tsne_model

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4     -29.594002  60.112713       1        1  22.811980
0     -45.996586  11.891685       2        1  14.895945
5      20.368856  70.019325       3        1  14.868084
2      53.928768  31.704309       4        1  14.123763
3     -12.437127 -26.423267       5        1  13.302649
6       3.965971  21.797720       6        1  12.358246
1      37.524284 -16.517805       7        1   7.639333, topic_info=      Category          Freq       Term         Total  loglift  logprob
1809   Default  16638.000000  bloomberg  16638.000000  30.0000  30.0000
11001  Default   8779.000000       pete   8779.000000  29.0000  29.0000
15539  Default  20220.000000      trump  20220.000000  28.0000  28.0000
1270   Default  11043.000000     bernie  11043.000000  27.0000  27.0000
2453   Default   6219.000000  buttigieg   6219.000000  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
11737   Topic7    240.553525        pro    469.530003   1.9031  -5.9300
8006    Topic7    252.520498       iran    667.471375   1.5999  -5.8815
11523   Topic7    359.153315  president   7785.355961  -0.5044  -5.5292
11277   Topic7    262.997045     policy   1085.210645   1.1545  -5.8408
1270    Topic7    302.204469     bernie  11043.419366  -1.0266  -5.7019

[523 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6          4  0.990922        abbas
27         5  0.997566     abortion
30         5  0.978944    abortions
46         4  0.994459        abuse
47         4  0.995824  abuse power
...      ...       ...          ...
17299      4  0.018099         york
17299      5  0.056881         york
17299      6  0.620522         york
17299      7  0.040075         york
17300      6  0.996875   york mayor

[1269 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 3, 4, 7, 2])

In [ ]:
tsne_model = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='tsne')
tsne_model